# Model input ERP format preparation 

In this notebook: 
- Necessary inputs
- Read all epochs
- Function to create dataframe with average mismatch response for all participants (needs to be transformed to function)
- Formatting dataframe as suitable model input

## Imports

In [1]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
from ipywidgets import IntSlider, Output
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from math import nan
from scipy import stats

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import sys
sys.path.insert(0, '../eegyolk') # path to helper functions
import helper_functions as hf # library useful for eeg and erp data cleaning
import initialization_functions #library to import data
import epod_helper

In [2]:
metadata = pd.read_csv('metadata.csv', sep = ',')

In [3]:
metadata

,eeg_file,ParticipantID,test,sex,age_months,age_months_days,dyslexic_parent,Group_AccToParents,eeg_path,path_epoch,epoch_file
0,101a,101,a,m,20,20;22,m,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,epochs,101a_epo.fif
1,101b,101,b,m,23,23;16,m,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,epochs,101b_epo.fif
2,102a,102,a,f,20,20;27,Nee,Control,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,epochs,102a_epo.fif
3,102b,102,b,f,23,23;16,Nee,Control,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,epochs,102b_epo.fif
4,103a,103,a,f,20,20;23,m,At risk,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,epochs,103a_epo.fif
...,...,...,...,...,...,...,...,...,...,...,...
57,134a,134,a,m,19,19;20,Nee,Control,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,epochs,134a_epo.fif
58,134b,134,b,m,23,23;7,Nee,Control,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,epochs,134b_epo.fif
59,135a,135,a,f,18,18;21,Nee,Control,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,epochs,135a_epo.fif
60,135b,135,b,f,22,22;8,Nee,Control,F:\Stage\ePODIUM\Data\ePodium_projectfolder\Da...,epochs,135b_epo.fif


## Read all epochs from files

The function below loads all filtered epochs from the dataframe which contains the metadata and the epochs filepaths + filenames. The epochs are arrays for each stimuli with a time interval of -0.3 to 0.7.

In [4]:
def read_filtered_data(metadata):
    epochs = []
    for index, file in metadata.iterrows():
        print(f"Checking out file: {file['epoch_file']}")
        path = os.path.join(file['path_epoch'], file['epoch_file'])
        epoch = mne.read_epochs(path, preload=False)
        epochs.append(epoch)
    return epochs

In [5]:
epochs = read_filtered_data(metadata)

Checking out file: 101a_epo.fif
Reading F:\Stage\ePODIUM\eegyolk\dyslexiaprediction_nadine\epochs\101a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2266 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 101b_epo.fif
Reading F:\Stage\ePODIUM\eegyolk\dyslexiaprediction_nadine\epochs\101b_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2440 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 102a_epo.fif
Reading F:\Stage\ePODIUM\eegyolk\dyslexiaprediction_nadine\epochs\102a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2389 matching events found
No baseline correction applied
0 p

Checking out file: 115a_epo.fif
Reading F:\Stage\ePODIUM\eegyolk\dyslexiaprediction_nadine\epochs\115a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
1220 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 115b_epo.fif
Reading F:\Stage\ePODIUM\eegyolk\dyslexiaprediction_nadine\epochs\115b_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
1220 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 116a_epo.fif
Reading F:\Stage\ePODIUM\eegyolk\dyslexiaprediction_nadine\epochs\116a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2266 matching events found
No baseline correction applied
0 p

Checking out file: 129b_epo.fif
Reading F:\Stage\ePODIUM\eegyolk\dyslexiaprediction_nadine\epochs\129b_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2302 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 130a_epo.fif
Reading F:\Stage\ePODIUM\eegyolk\dyslexiaprediction_nadine\epochs\130a_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2389 matching events found
No baseline correction applied
0 projection items activated
Checking out file: 130b_epo.fif
Reading F:\Stage\ePODIUM\eegyolk\dyslexiaprediction_nadine\epochs\130b_epo.fif ...
    Found the data of interest:
        t =    -299.80 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
2440 matching events found
No baseline correction applied
0 p

## Create pandas dataframe with the average difference between standard and deviant responses

The function below needs `metadata`, the loaded `epochs` and the definition of the standard and deviant events as input. You should define your standard and deviant events as an array. In the function `input_mmr_prep` it's important to know that the assumption is made that the deviant follows after a standard event. Therefore the deviant belonging to the standard is the  standard event number + 1. Make sure your events are numbered like this, else the function won't calculate the mismatch response.  

In [6]:
# define the events for standard and deviant
standard_events = [2,5,8,11]
deviant_events = [3,6,9,12]

def input_mmr_prep(metadata, epochs, standard_events): 
    # create dataframe with expected columns 
    df = pd.DataFrame(columns=["eeg_file", "paradigm", "channel", "mean"])

    # loop over all participants
    for i in range(len(metadata['eeg_file'])):
        
        # loop over every paradigm per participant 
        for j in standard_events: 
            paradigm = j
            # select the standard and deviant for a specific sequence and calculate the evoked response
            std_evoked = epochs[i][j].average() 
            dev_evoked = epochs[i][j+1].average()
            
            # calculate the mismatch response between standard and deviant evoked
            evoked_diff = mne.combine_evoked([std_evoked, dev_evoked], weights=[1, -1])
            
            # get a list of all channels
            chnames_list = evoked_diff.info['ch_names']
            
            # compute for every channel the distance mean of the mismatch line
            for channel in chnames_list: 
                chnames = mne.pick_channels(evoked_diff.info['ch_names'], include=[channel])
                roi_dict = dict(left_ROI=chnames) # unfortunately combine_channels only takes a dictionary as input
                roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
                mmr = roi_evoked.to_data_frame()
                mmr_avg = mmr['left_ROI'].mean()
                mmr_std = mmr['left_ROI'].std()
                mmr_skew = mmr['left_ROI'].skew()
                mmr_var = mmr['left_ROI'].var()
                mmr_kurt = mmr['left_ROI'].kurtosis()
                
                df = df.append({'eeg_file': metadata['eeg_file'][i], 'paradigm' : paradigm, 'channel': channel, 'mean' :  mmr_avg, 'std' : mmr_std, 'skew' : mmr_skew, 'kurt' : mmr_kurt, 'var' : mmr_var}, ignore_index=True) 
    return df

In [ ]:
df = input_mmr_prep(metadata, epochs, standard_events)

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoke

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoke

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoke

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoke

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoke

...

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoke

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoke

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoke

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoke

<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
<ipython-input-6-60b7c6f8b01a>:29: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x00000277A9BF9C10>, axis=0)".
  roi_evoke

In [ ]:
df

## Transpose dataframe into combination of paradigm and channel per participant

We now want a single row for every participant containing the paradigm and corresponding channels. The code below generates this dataframe. 

In [ ]:
# transformation of the dataframe
df = df.pivot(index='eeg_file', columns=['paradigm','channel'])
df

In [ ]:
df.columns = ['_'.join(str(s).strip() for s in col if s) for col in df.columns]
df

In [ ]:
df.reset_index(inplace=True)
df

## Merge and safe dataframe

We still need to merge some of the metadata into the dataframe, so we have the information of the age, gender and label of the participant. 

In [ ]:
df = pd.merge(df, metadata, on='eeg_file')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df

Drop some unnecessary columns. 

In [ ]:
df = df.drop(['eeg_file','age_months_days',
       'dyslexic_parent', 'eeg_path', 'path_epoch',
       'epoch_file'], axis =1)

Every participant has now two rows, one for each test. We now want a single row for each participant, since the participant already has experience with the test and therefore the test at time b might differ from the test atime a. To avoid bias in the data, both tests will be appended to the participant into one row. 

In [ ]:
a = df.loc[df['test'] == 'a']
b = df.loc[df['test'] == 'b']
a = a.drop(['test'],axis=1)
b = b.drop(['test','sex','age_months','Group_AccToParents'], axis=1)

In [ ]:
df = pd.merge(a, b, on='ParticipantID',suffixes=("_a", "_b"))
df = df.drop(['ParticipantID'],axis=1)

In [ ]:
df['sex'] = np.where((df['sex']=='m'), 1,0)
df['Group_AccToParents'] = np.where((df['Group_AccToParents']=='At risk'), 1,0)

In [ ]:
first = df.pop('Group_AccToParents')
df.insert(0, 'Group_AccToParents', first)

To remove some outliers on the data, the z score is calculated. 

In [ ]:
df[(np.abs(stats.zscore(df)) < 5).all(axis=1)] # z score 

In [ ]:
df.to_csv('df_avg_mmr.csv', index=False) # safe dataframe

## PCA analysis on feature reduction 

In [ ]:
X = df.drop('Group_AccToParents',1)
y = df['Group_AccToParents']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
explained_variance = pca.explained_variance_ratio_

In [ ]:
explained_variance

In [ ]:
classifier = RandomForestClassifier(max_depth=2, random_state=0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))

In [ ]:
X.shape